# Cirq Introduction: Quantum Random Number Generator

This notebook introduces basic Cirq concepts: qubits, gates vs. operations, circuits, simulation, and collecting measurement statistics. We'll build a simple quantum random number generator (QRNG) using a Hadamard gate and a measurement.

## 1. Environment Setup
If Cirq isn't installed in your environment you can uncomment and run the next cell. (It's often pre-installed in curated quantum environments.)

In [ ]:
# Optional install (uncomment if needed)
# %pip install cirq --quiet

## 2. Imports
We first import Cirq.

In [ ]:
import cirq

## 3. Defining the QRNG Program
We create a function returning a sequence of Cirq operations. Note the distinction: a *Gate* is a template; an *Operation* applies that gate to specific qubits.

In [ ]:
from typing import Sequence

def qrng() -> Sequence[cirq.Operation]:
    a = cirq.NamedQubit("a")  # Could also use LineQubit, GridQubit, etc.
    return [
        cirq.H.on(a),            # Put |0> into equal superposition |+>
        cirq.measure(a, key='result')  # Measure in Z basis -> random 0/1
    ]

## 4. Building a Circuit
A `cirq.Circuit` organizes moments (time steps). We'll print the raw operations and the formatted circuit.

In [ ]:
program = qrng()
print('Program operations:')
for op in program:
    print('  ', op)

circuit = cirq.Circuit(program)
print('Circuit:')
print(circuit)

## 5. Visualizing the Circuit (cirq.vis)

Besides printing a textual diagram, Cirq can render richer visualizations. Below we try:

1. `cirq.vis` (newer visualization APIs, if present in this Cirq version).
2. Fallback to the older SVG helper (`cirq.contrib.svg.SVGCircuit`).
3. Final fallback: print the text diagram again.

Run the next cell to attempt these visualizations.

In [ ]:
# Attempt richer visualization pathways for the freshly built `circuit`.
visualized = False
try:
    import cirq.vis as cvis
    if hasattr(cvis, 'Circuit') and callable(getattr(cvis, 'Circuit')):
        display(cvis.Circuit(circuit))
        visualized = True
    elif hasattr(cvis, 'render_circuit'):
        cvis.render_circuit(circuit)
        visualized = True
except Exception as e:
    print("cirq.vis not available or failed:", e)

if not visualized:
    try:
        from cirq.contrib.svg import SVGCircuit
        display(SVGCircuit(circuit))
        visualized = True
    except Exception as e:
        print("SVG contrib visualization not available:", e)

if not visualized:
    print("Plain text diagram fallback (already printed above):\n")
    print(circuit)

## 6. Simulation

We run the circuit multiple times (shots) to build a histogram of outcomes.

In [ ]:
simulator = cirq.Simulator()
result = simulator.run(circuit, repetitions=1000)
hist = result.histogram(key='result')
print('Raw counts:', hist)

## 7. Visualizing the Measurement Distribution

Ideally the counts are close to 50/50 for 0 and 1 (quantum randomness). We will visualize the results using matplotlib.

In [ ]:
import matplotlib.pyplot as plt

plt.bar(hist.keys(), hist.values(), color=['#4C72B0', '#55A868'])
plt.xticks([0,1])
plt.xlabel('Measured value')
plt.ylabel('Counts')
plt.title('QRNG Output Distribution (1000 shots)')
plt.show()

## 8. Next Steps

You now know how to: create a qubit, apply a gate, build a circuit, visualize it, and simulate measurements. Continue with the teleportation notebook to explore multi-qubit circuits and classical control.